# FEC Analysis

## Objectives
1. Identify a suitable threshold for binarizing
2. Identify a suitable threshold for Conditioned Response

## Approach for identifying threshold
1. Blur ROI using 3x3 average filters to avoid salt and pepper like single pixel noise
2. Plot histogram of the intensities of the blurred roi
3. Use a gaussian filter to smoothen over the histogram
4. Identify 'inflection_points' as the intensity at whicch the curve is about to switch from negative slope to positive slope
5. Steps 1-4 are repeated for all frames in the 'PRE' period. The median inflection point is chosen as the threshold for binarizing all frames within a trial

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
import cv2
from scipy.ndimage import gaussian_filter1d
import math

In [ ]:
path = "../../hrishikeshn/BehaviorRawData/G405/G405_An1_11/"
file_name = path + "001.tiff"
roi = (375, 95, 405, 245)
sigma = 0.75  # standard deviation of gaussian filter
n_bins = 50  # number of bins for intensity histogram

In [ ]:
def get_threshold(frame_stack, roi):
    x_min, y_min, x_max, y_max = roi
    inflection_points = []

    for f, frame in enumerate(frame_stack):
        bin_line = frame[0, :]
        data_line = ("".join(([chr(x) for x in bin_line]))).rstrip()
        tokens = data_line.split(",")

        if tokens[10] == "PRE_":
            eye_roi = frame[y_min:y_max, x_min:x_max]
            blurred_roi = cv2.blur(eye_roi, (3, 3))
            bins = np.linspace(0, 255, n_bins)
            counts, bins = np.histogram(blurred_roi, bins=bins, density=True)
            smoothed_counts = gaussian_filter1d(counts, sigma)
            smoothed_d1 = np.gradient(smoothed_counts)
            inflection_point_indices = np.where(np.diff(np.sign(smoothed_d1)) > 1.5)[0]
            if np.any(inflection_point_indices):
                inflection_points.append(bins[inflection_point_indices[0]])
            else:
                inflection_points.append(np.nan)

        else:
            break

    threshold = np.nanmedian(inflection_points)
    print(threshold)
    if math.isnan(threshold):
        print("Obtained Nan value for binarizing threshold")
    return threshold

In [ ]:
def plot_images(frame_stack, roi, threshold):
    x_min, y_min, x_max, y_max = roi

    for f, frame in enumerate(frame_stack):
        if f%50 == 0:
            eye_roi = frame[y_min:y_max, x_min:x_max]
            blurred_roi = cv2.blur(eye_roi, (3, 3))
            binarized_roi = blurred_roi > threshold

            bins = np.linspace(0, 255, n_bins)
            counts, bins = np.histogram(blurred_roi, bins=bins, density=True)
            smoothed_counts = gaussian_filter1d(counts, sigma)
            cum_counts = np.cumsum(counts) * (bins[1] - bins[0])

            fig = plt.figure(figsize=(7, 6))
            plt.suptitle(f"frame #{f}")
            spec = fig.add_gridspec(2, 6)

            axa = fig.add_subplot(spec[0, 0:2])
            axa.imshow(eye_roi, cmap="gray")
            axa.set_title("original")

            axb = fig.add_subplot(spec[0, 2:4])
            axb.imshow(blurred_roi, cmap="gray")
            axb.set_title("blurred")

            axc = fig.add_subplot(spec[0, 4:6])
            axc.imshow(binarized_roi, cmap="gray")
            axc.set_title("binarized")

            axd = fig.add_subplot(spec[1, 0:3])
            axd.hist(blurred_roi, bins=bins, density=True)
            axd.plot(bins[1:], smoothed_counts, label="gaussian smoothening")
            axd.axvline(threshold, color="r", label="threshold")
            axd.set_xlabel("intensity")
            axd.set_ylabel("probability")
            axd.set_title("blurred")
            axd.legend()

            axe = fig.add_subplot(spec[1, 3:6])
            axe.plot(bins[1:], cum_counts)
            axe.axvline(threshold, color="r", label="threshold")
            axe.set_xlabel("intensity")
            axe.set_ylabel("cumulative probability")
            axe.set_title("blurred")
            axe.legend()

In [ ]:
frame_stack = io.imread(file_name)
threshold = get_threshold(frame_stack, roi)
plot_images(frame_stack, roi, threshold)

## What next?
1. Look at Conditioned Responses
2. Test out the negated logic for white-eyed mice